In [1]:
%reload_ext autotime
import geopandas as gpd
import pandas as pd
from glob import glob
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, root_mean_squared_error
from coastsat import SDS_transects
pd.options.plotting.backend = "plotly"

In [2]:
# Transects, origin is landward. Has beach_slope
transects = gpd.read_file("transects_extended.geojson")
transects.set_index("id", inplace=True)
transects

site_id  orientation  along_dist  along_dist_norm  beach_slope  \
id                                                                             
aus0001-0000  aus0001   104.347648    0.000000         0.000000        0.085   
aus0001-0001  aus0001    93.495734   98.408334         0.002935        0.050   
aus0001-0002  aus0001    82.069341  198.408334         0.005918        0.050   
aus0001-0003  aus0001    81.192757  298.402523         0.008900        0.055   
aus0001-0004  aus0001    81.065473  398.402523         0.011882        0.075   
...               ...          ...         ...              ...          ...   
ber0002-0009  ber0002          NaN         NaN              NaN        0.080   
ber0002-0010  ber0002          NaN         NaN              NaN        0.085   
ber0002-0011  ber0002          NaN         NaN              NaN        0.105   
ber0002-0012  ber0002          NaN         NaN              NaN        0.105   
ber0002-0013  ber0002          NaN         NaN              NaN        0.055   

                 cil     ciu     trend  n_points  n_points_nonan  r2_score  \
id                                                                           
aus0001-0000  0.0545  0.2000 -1.441081     767.0           428.0  0.168420   
aus0001-0001  0.0387  0.0640 -1.037105     767.0           569.0  0.097874   
aus0001-0002  0.0428  0.0647 -0.680019     767.0           588.0  0.053927   
aus0001-0003  0.0480  0.0659 -0.405198     767.0           598.0  0.023412   
aus0001-0004  0.0614  0.0922 -0.090025     767.0           608.0  0.001277   
...              ...     ...       ...       ...             ...       ...   
ber0002-0009  0.0638  0.1065  0.088578     223.0           192.0  0.006440   
ber0002-0010  0.0673  0.1096  0.028300     223.0           190.0  0.000497   
ber0002-0011  0.0797  0.1462  0.057356     223.0           187.0  0.002423   
ber0002-0012  0.0835  0.1428 -0.010553     223.0           186.0  0.000081   
ber0002-0013  0.0491  0.0674  0.157248     223.0           177.0  0.014649   

                    mae          mse       rmse   intercept ERODIBILITY  \
id                                                                        
aus0001-0000  28.102591  1263.560863  35.546601  179.085729        None   
aus0001-0001  25.419324  1033.770813  32.152306  212.247788        None   
aus0001-0002  22.632907   838.007507  28.948359  205.106151        None   
aus0001-0003  20.749758   698.653187  26.432048  191.745881        None   
aus0001-0004  19.889328   655.810616  25.608800  175.092121        None   
...                 ...          ...        ...         ...         ...   
ber0002-0009   6.645872    70.165869   8.376507  128.134846        None   
ber0002-0010   7.682399    92.744157   9.630377  128.837353        None   
ber0002-0011   7.512558    79.234887   8.901398  129.482491        None   
ber0002-0012   7.210548    79.289497   8.904465  131.010602        None   
ber0002-0013   7.547493    95.120607   9.752979  136.403294        None   

                                                       geometry  
id                                                               
aus0001-0000  LINESTRING (153.26555 -24.7007, 153.26938 -24....  
aus0001-0001  LINESTRING (153.26525 -24.7019, 153.2692 -24.7...  
aus0001-0002  LINESTRING (153.26539 -24.70316, 153.26931 -24...  
aus0001-0003  LINESTRING (153.26555 -24.70408, 153.26945 -24...  
aus0001-0004  LINESTRING (153.2657 -24.70497, 153.26961 -24....  
...                                                         ...  
ber0002-0009  LINESTRING (-64.82204 32.25336, -64.82017 32.2...  
ber0002-0010  LINESTRING (-64.82143 32.25361, -64.82029 32.2...  
ber0002-0011  LINESTRING (-64.82118 32.25369, -64.82004 32.2...  
ber0002-0012  LINESTRING (-64.82049 32.25382, -64.82022 32.2...  
ber0002-0013  LINESTRING (-64.82022 32.25384, -64.81995 32.2...  

[147216 rows x 17 columns]

In [3]:
vos_files = pd.Series(
    sorted(glob("csv_run7/*/time_series_tidally_corrected.csv"))
)
vos_files = vos_files[~vos_files.str.contains("nzd")]
vos_files

0       csv_run7/aus0001/time_series_tidally_corrected...
1       csv_run7/aus0002/time_series_tidally_corrected...
2       csv_run7/aus0003/time_series_tidally_corrected...
3       csv_run7/aus0004/time_series_tidally_corrected...
4       csv_run7/aus0005/time_series_tidally_corrected...
                              ...                        
3066    csv_run7/usa_HI_0053/time_series_tidally_corre...
3067    csv_run7/usa_HI_0054/time_series_tidally_corre...
3068    csv_run7/usa_HI_0055/time_series_tidally_corre...
3069    csv_run7/usa_HI_0056/time_series_tidally_corre...
3070    csv_run7/usa_HI_0057/time_series_tidally_corre...
Length: 2511, dtype: object

In [4]:
my_files = pd.Series(
    sorted(glob("data/*/transect_time_series_tidally_corrected.csv"))
)
my_files

0      data/ber0001/transect_time_series_tidally_corr...
1      data/ber0002/transect_time_series_tidally_corr...
2      data/nzd0001/transect_time_series_tidally_corr...
3      data/nzd0002/transect_time_series_tidally_corr...
4      data/nzd0003/transect_time_series_tidally_corr...
                             ...                        
559    data/nzd0559/transect_time_series_tidally_corr...
560    data/nzd0560/transect_time_series_tidally_corr...
561    data/nzd0561/transect_time_series_tidally_corr...
562    data/nzd0562/transect_time_series_tidally_corr...
563    data/nzd0563/transect_time_series_tidally_corr...
Length: 564, dtype: object

In [5]:
sar_files = pd.Series(sorted(glob("data/sar*/transect_time_series.csv")))
sar_files

0       data/sar0001/transect_time_series.csv
1       data/sar0002/transect_time_series.csv
2       data/sar0003/transect_time_series.csv
3       data/sar0004/transect_time_series.csv
4       data/sar0005/transect_time_series.csv
                        ...                  
2536    data/sar2537/transect_time_series.csv
2537    data/sar2538/transect_time_series.csv
2538    data/sar2539/transect_time_series.csv
2539    data/sar2540/transect_time_series.csv
2540    data/sar2541/transect_time_series.csv
Length: 2541, dtype: object

In [6]:
files = pd.concat([vos_files, my_files, sar_files])
files

0       csv_run7/aus0001/time_series_tidally_corrected...
1       csv_run7/aus0002/time_series_tidally_corrected...
2       csv_run7/aus0003/time_series_tidally_corrected...
3       csv_run7/aus0004/time_series_tidally_corrected...
4       csv_run7/aus0005/time_series_tidally_corrected...
                              ...                        
2536                data/sar2537/transect_time_series.csv
2537                data/sar2538/transect_time_series.csv
2538                data/sar2539/transect_time_series.csv
2539                data/sar2540/transect_time_series.csv
2540                data/sar2541/transect_time_series.csv
Length: 5616, dtype: object

In [7]:
f = files[files.str.contains("ber0001")].iloc[0]
# despiked_filename = f.replace(".csv", "_tidally_corrected.csv")
df = pd.read_csv(f)
df.dates = pd.to_datetime(df.dates)
df.set_index("dates", inplace=True)
display(df.columns)
import matplotlib.pyplot as plt

transect_id = "ber0001-0002"


def custom_mean(window):
    return window[window.between(window.quantile(0.25), window.quantile(0.75))].mean()


pd.DataFrame(
    {
        "raw": df[transect_id],
        "rolling 90d mean": df[transect_id].rolling("90d", min_periods=1).mean(),
        "rolling 180d mean": df[transect_id].rolling("180d", min_periods=1).mean(),
        "rolling 90d custom mean": df[transect_id]
        .rolling("90d", min_periods=1)
        .apply(custom_mean),
        "rolling 180d custom mean": df[transect_id]
        .rolling("180d", min_periods=1)
        .apply(custom_mean),
        # "rolling 365d": df[transect_id].rolling("365d", min_periods=1).mean(),
    },
    index=df.index,
).plot()

Index(['ber0001-0000', 'ber0001-0001', 'ber0001-0002', 'ber0001-0003',
       'ber0001-0004', 'ber0001-0005', 'ber0001-0006', 'ber0001-0007',
       'ber0001-0008', 'ber0001-0009', 'satname'],
      dtype='object')

In [8]:
df = pd.read_csv("data/sar0939/transect_time_series.csv")
df.dates = pd.to_datetime(df.dates)
df.set_index("dates", inplace=True)
(df["sar0939-0000"] - 93).plot()

In [9]:
def despike(chainage, threshold=40):
    chainage = chainage.dropna()
    chainage, dates = SDS_transects.identify_outliers(
        chainage.tolist(), chainage.index.tolist(), threshold
    )
    return pd.Series(chainage, index=dates)


def get_trends(f):
    df = pd.read_csv(f)
    try:
        df.dates = pd.to_datetime(df.dates)
    except:
        print(f)
    if "sar" in f or "ber" in f:
        smoothed_filename = f.replace(".csv", "_smoothed.csv")
        try:
            df = pd.read_csv(smoothed_filename)
            df.dates = pd.to_datetime(df.dates)
        except:
            df.dates = pd.to_datetime(df.dates)
            df.set_index("dates", inplace=True)
            satname = df.satname
            df = df.drop(columns="satname").apply(despike, axis=0)
            df["satname"] = satname
            df.reset_index(names="dates").to_csv(
                f.replace(".csv", "_despiked.csv"), index=False
            )
            for transect_id in df.drop(columns="satname").columns:
                df[transect_id] = df[transect_id].rolling("180d", min_periods=1).mean()
            df.reset_index(names="dates", inplace=True)
            df.to_csv(f.replace(".csv", "_smoothed.csv"), index=False)
    df.index = (df.dates - df.dates.min()).dt.days / 365.25
    df.drop(columns=["dates", "satname", "Unnamed: 0"], inplace=True, errors="ignore")
    trends = []
    for transect_id in df.columns:
        sub_df = df[transect_id].dropna()
        if not len(sub_df):
            continue
        x = sub_df.index.to_numpy().reshape(-1, 1)
        y = sub_df
        linear_model = LinearRegression().fit(x, y)
        pred = linear_model.predict(x)
        trends.append(
            {
                "transect_id": transect_id,
                "trend": linear_model.coef_[0],
                "intercept": linear_model.intercept_,
                "n_points": len(df[transect_id]),
                "n_points_nonan": len(sub_df),
                "r2_score": r2_score(y, pred),
                "mae": mean_absolute_error(y, pred),
                "mse": mean_squared_error(y, pred),
                "rmse": root_mean_squared_error(y, pred),
            }
        )
    return pd.DataFrame(trends)


# trends = get_trends(sar_files.iloc[-1]).set_index("transect_id")
trends = pd.concat(process_map(get_trends, my_files)).set_index("transect_id")
len(trends)

  0%|          | 0/564 [00:00<?, ?it/s]

32373

In [10]:
trends[trends.n_points_nonan > 10].sort_values("r2_score")

trend   intercept  n_points  n_points_nonan      r2_score  \
transect_id                                                                   
nzd0431-0015  -0.000036  366.186580       540             391  3.259719e-10   
nzd0096-0005   0.000075  367.077323       293             197  6.685417e-10   
nzd0252-0022  -0.000087  311.449858       344             237  1.223139e-09   
nzd0359-0072   0.000048  349.976585       493             353  1.902363e-09   
nzd0404-0171  -0.000095  316.349783       552             454  2.773332e-09   
...                 ...         ...       ...             ...           ...   
nzd0121-0001  15.046272  275.304171       172             108  9.436296e-01   
nzd0121-0002  15.309330  250.088130       172             111  9.438529e-01   
nzd0121-0003  14.527931  243.590620       172             116  9.444384e-01   
nzd0121-0005  14.351756  207.715488       172             115  9.495681e-01   
nzd0121-0006  14.267696  190.486846       172             115  9.501103e-01   

                    mae         mse       rmse  
transect_id                                     
nzd0431-0015  11.601473  215.766647  14.688997  
nzd0096-0005  17.207332  457.713592  21.394242  
nzd0252-0022  14.626887  339.816075  18.434101  
nzd0359-0072   6.248942   71.303735   8.444154  
nzd0404-0171  10.998206  190.340365  13.796390  
...                 ...         ...        ...  
nzd0121-0001  21.794963  787.151299  28.056217  
nzd0121-0002  22.016239  777.948935  27.891736  
nzd0121-0003  19.819520  676.388746  26.007475  
nzd0121-0005  20.341560  622.917772  24.958321  
nzd0121-0006  19.631684  592.746482  24.346385  

[32372 rows x 8 columns]

In [11]:
trends[trends.index.str.startswith("ber")]

trend   intercept  n_points  n_points_nonan      r2_score  \
transect_id                                                                  
ber0001-0000 -0.462662  172.942131       217             217  4.091450e-01   
ber0001-0001 -0.377595  164.045559       217             217  4.158845e-01   
ber0001-0002 -0.237611  162.250470       217             217  1.909726e-01   
ber0001-0003 -0.231374  162.928497       217             217  1.937340e-01   
ber0001-0004 -0.244762  163.289489       217             217  1.915469e-01   
ber0001-0005 -0.203432  160.599356       217             217  1.253129e-01   
ber0001-0006 -0.104889  157.413329       217             217  3.142298e-02   
ber0001-0007 -0.156261  160.676732       217             217  9.869806e-02   
ber0001-0008 -0.392455  166.533154       217             217  5.216501e-01   
ber0001-0009 -0.318971  165.491387       217             217  4.113697e-01   
ber0002-0000  0.342658  143.257489       199             197  1.280613e-01   
ber0002-0001 -0.044439  124.193301       199             198  4.000118e-03   
ber0002-0002 -0.004414  124.217151       199             198  3.776566e-05   
ber0002-0003 -0.088823  126.080845       199             198  1.557075e-02   
ber0002-0004 -0.035432  123.893841       199             197  2.965913e-03   
ber0002-0005 -0.000366  123.961417       199             197  3.297592e-07   
ber0002-0006  0.022537  124.965195       199             197  1.209053e-03   
ber0002-0007  0.043525  124.593984       199             197  3.950463e-03   
ber0002-0008  0.022019  126.373199       199             197  9.159122e-04   
ber0002-0009  0.159612  127.283966       199             199  4.146091e-02   
ber0002-0010  0.071946  128.858980       199             197  1.073030e-02   
ber0002-0011  0.081426  129.347401       199             198  1.182336e-02   
ber0002-0012  0.032630  130.592921       199             197  2.116556e-03   
ber0002-0013  0.184478  136.265615       199             199  4.977507e-02   

                   mae        mse      rmse  
transect_id                                  
ber0001-0000  3.194084  19.180097  4.379509  
ber0001-0001  2.616221  12.425027  3.524915  
ber0001-0002  2.813023  14.840370  3.852320  
ber0001-0003  2.681119  13.823584  3.718008  
ber0001-0004  3.092245  15.688707  3.960897  
ber0001-0005  3.317711  17.923198  4.233580  
ber0001-0006  3.673513  21.040842  4.587030  
ber0001-0007  2.977771  13.835097  3.719556  
ber0001-0008  2.195485   8.763265  2.960281  
ber0001-0009  2.291076   9.033018  3.005498  
ber0002-0000  5.146820  43.763337  6.615386  
ber0002-0001  3.990024  27.431579  5.237516  
ber0002-0002  4.089086  28.778088  5.364521  
ber0002-0003  4.183383  27.826626  5.275095  
ber0002-0004  3.923212  23.521058  4.849851  
ber0002-0005  3.789184  22.626797  4.756763  
ber0002-0006  3.915333  23.384722  4.835775  
ber0002-0007  4.189945  26.621844  5.159636  
ber0002-0008  4.439873  29.475248  5.429111  
ber0002-0009  4.326802  33.378983  5.777455  
ber0002-0010  4.357300  26.732016  5.170301  
ber0002-0011  4.779105  31.534469  5.615556  
ber0002-0012  4.463024  28.118400  5.302679  
ber0002-0013  4.886920  36.819225  6.067885

In [12]:
trends.describe()

trend     intercept      n_points  n_points_nonan      r2_score  \
count  32373.000000  32373.000000  32373.000000    32373.000000  3.237300e+04   
mean       0.119917    351.345014    440.309981      325.687981  1.481745e-01   
std        1.880075     54.966560    153.958435      118.735401  1.905990e-01   
min      -40.158498   -174.937352    102.000000       10.000000  3.259719e-10   
25%       -0.393352    327.998641    337.000000      245.000000  1.418003e-02   
50%        0.073520    349.046157    422.000000      319.000000  6.543375e-02   
75%        0.572345    372.134647    563.000000      415.000000  2.051896e-01   
max       27.751623   1138.442745    814.000000      714.000000  9.501103e-01   

                mae           mse          rmse  
count  32373.000000  32373.000000  32373.000000  
mean      12.561924    362.984323     16.019111  
std        8.364891    805.676842     10.313860  
min        2.195485      8.763265      2.960281  
25%        8.244322    111.819940     10.574495  
50%       10.321347    172.876232     13.148241  
75%       13.746771    304.449683     17.448487  
max      118.288420  20472.735372    143.082967

In [13]:
(transects.trend - trends.trend).describe()

count    32395.000000
mean         0.002205
std          0.049364
min         -1.555044
25%         -0.005849
50%          0.000000
75%          0.009200
max          4.203379
Name: trend, dtype: float64

In [14]:
transects.update(trends.drop_duplicates())

In [15]:
transects

site_id  orientation  along_dist  along_dist_norm  beach_slope  \
id                                                                             
aus0001-0000  aus0001   104.347648    0.000000         0.000000        0.085   
aus0001-0001  aus0001    93.495734   98.408334         0.002935        0.050   
aus0001-0002  aus0001    82.069341  198.408334         0.005918        0.050   
aus0001-0003  aus0001    81.192757  298.402523         0.008900        0.055   
aus0001-0004  aus0001    81.065473  398.402523         0.011882        0.075   
...               ...          ...         ...              ...          ...   
ber0002-0009  ber0002          NaN         NaN              NaN        0.080   
ber0002-0010  ber0002          NaN         NaN              NaN        0.085   
ber0002-0011  ber0002          NaN         NaN              NaN        0.105   
ber0002-0012  ber0002          NaN         NaN              NaN        0.105   
ber0002-0013  ber0002          NaN         NaN              NaN        0.055   

                 cil     ciu     trend  n_points  n_points_nonan  r2_score  \
id                                                                           
aus0001-0000  0.0545  0.2000 -1.441081     767.0           428.0  0.168420   
aus0001-0001  0.0387  0.0640 -1.037105     767.0           569.0  0.097874   
aus0001-0002  0.0428  0.0647 -0.680019     767.0           588.0  0.053927   
aus0001-0003  0.0480  0.0659 -0.405198     767.0           598.0  0.023412   
aus0001-0004  0.0614  0.0922 -0.090025     767.0           608.0  0.001277   
...              ...     ...       ...       ...             ...       ...   
ber0002-0009  0.0638  0.1065  0.159612     199.0           199.0  0.041461   
ber0002-0010  0.0673  0.1096  0.071946     199.0           197.0  0.010730   
ber0002-0011  0.0797  0.1462  0.081426     199.0           198.0  0.011823   
ber0002-0012  0.0835  0.1428  0.032630     199.0           197.0  0.002117   
ber0002-0013  0.0491  0.0674  0.184478     199.0           199.0  0.049775   

                    mae          mse       rmse   intercept ERODIBILITY  \
id                                                                        
aus0001-0000  28.102591  1263.560863  35.546601  179.085729        None   
aus0001-0001  25.419324  1033.770813  32.152306  212.247788        None   
aus0001-0002  22.632907   838.007507  28.948359  205.106151        None   
aus0001-0003  20.749758   698.653187  26.432048  191.745881        None   
aus0001-0004  19.889328   655.810616  25.608800  175.092121        None   
...                 ...          ...        ...         ...         ...   
ber0002-0009   4.326802    33.378983   5.777455  127.283966        None   
ber0002-0010   4.357300    26.732016   5.170301  128.858980        None   
ber0002-0011   4.779105    31.534469   5.615556  129.347401        None   
ber0002-0012   4.463024    28.118400   5.302679  130.592921        None   
ber0002-0013   4.886920    36.819225   6.067885  136.265615        None   

                                                       geometry  
id                                                               
aus0001-0000  LINESTRING (153.26555 -24.7007, 153.26938 -24....  
aus0001-0001  LINESTRING (153.26525 -24.7019, 153.2692 -24.7...  
aus0001-0002  LINESTRING (153.26539 -24.70316, 153.26931 -24...  
aus0001-0003  LINESTRING (153.26555 -24.70408, 153.26945 -24...  
aus0001-0004  LINESTRING (153.2657 -24.70497, 153.26961 -24....  
...                                                         ...  
ber0002-0009  LINESTRING (-64.82204 32.25336, -64.82017 32.2...  
ber0002-0010  LINESTRING (-64.82143 32.25361, -64.82029 32.2...  
ber0002-0011  LINESTRING (-64.82118 32.25369, -64.82004 32.2...  
ber0002-0012  LINESTRING (-64.82049 32.25382, -64.82022 32.2...  
ber0002-0013  LINESTRING (-64.82022 32.25384, -64.81995 32.2...  

[147216 rows x 17 columns]

In [16]:
trends.columns, transects.columns, trends.columns.isin(transects.columns)

(Index(['trend', 'intercept', 'n_points', 'n_points_nonan', 'r2_score', 'mae',
        'mse', 'rmse'],
       dtype='object'),
 Index(['site_id', 'orientation', 'along_dist', 'along_dist_norm',
        'beach_slope', 'cil', 'ciu', 'trend', 'n_points', 'n_points_nonan',
        'r2_score', 'mae', 'mse', 'rmse', 'intercept', 'ERODIBILITY',
        'geometry'],
       dtype='object'),
 array([ True,  True,  True,  True,  True,  True,  True,  True]))

In [17]:
transects = transects.join(trends.loc[:, ~trends.columns.isin(transects.columns)])
transects

site_id  orientation  along_dist  along_dist_norm  beach_slope  \
id                                                                             
aus0001-0000  aus0001   104.347648    0.000000         0.000000        0.085   
aus0001-0001  aus0001    93.495734   98.408334         0.002935        0.050   
aus0001-0002  aus0001    82.069341  198.408334         0.005918        0.050   
aus0001-0003  aus0001    81.192757  298.402523         0.008900        0.055   
aus0001-0004  aus0001    81.065473  398.402523         0.011882        0.075   
...               ...          ...         ...              ...          ...   
ber0002-0009  ber0002          NaN         NaN              NaN        0.080   
ber0002-0010  ber0002          NaN         NaN              NaN        0.085   
ber0002-0011  ber0002          NaN         NaN              NaN        0.105   
ber0002-0012  ber0002          NaN         NaN              NaN        0.105   
ber0002-0013  ber0002          NaN         NaN              NaN        0.055   

                 cil     ciu     trend  n_points  n_points_nonan  r2_score  \
id                                                                           
aus0001-0000  0.0545  0.2000 -1.441081     767.0           428.0  0.168420   
aus0001-0001  0.0387  0.0640 -1.037105     767.0           569.0  0.097874   
aus0001-0002  0.0428  0.0647 -0.680019     767.0           588.0  0.053927   
aus0001-0003  0.0480  0.0659 -0.405198     767.0           598.0  0.023412   
aus0001-0004  0.0614  0.0922 -0.090025     767.0           608.0  0.001277   
...              ...     ...       ...       ...             ...       ...   
ber0002-0009  0.0638  0.1065  0.159612     199.0           199.0  0.041461   
ber0002-0010  0.0673  0.1096  0.071946     199.0           197.0  0.010730   
ber0002-0011  0.0797  0.1462  0.081426     199.0           198.0  0.011823   
ber0002-0012  0.0835  0.1428  0.032630     199.0           197.0  0.002117   
ber0002-0013  0.0491  0.0674  0.184478     199.0           199.0  0.049775   

                    mae          mse       rmse   intercept ERODIBILITY  \
id                                                                        
aus0001-0000  28.102591  1263.560863  35.546601  179.085729        None   
aus0001-0001  25.419324  1033.770813  32.152306  212.247788        None   
aus0001-0002  22.632907   838.007507  28.948359  205.106151        None   
aus0001-0003  20.749758   698.653187  26.432048  191.745881        None   
aus0001-0004  19.889328   655.810616  25.608800  175.092121        None   
...                 ...          ...        ...         ...         ...   
ber0002-0009   4.326802    33.378983   5.777455  127.283966        None   
ber0002-0010   4.357300    26.732016   5.170301  128.858980        None   
ber0002-0011   4.779105    31.534469   5.615556  129.347401        None   
ber0002-0012   4.463024    28.118400   5.302679  130.592921        None   
ber0002-0013   4.886920    36.819225   6.067885  136.265615        None   

                                                       geometry  
id                                                               
aus0001-0000  LINESTRING (153.26555 -24.7007, 153.26938 -24....  
aus0001-0001  LINESTRING (153.26525 -24.7019, 153.2692 -24.7...  
aus0001-0002  LINESTRING (153.26539 -24.70316, 153.26931 -24...  
aus0001-0003  LINESTRING (153.26555 -24.70408, 153.26945 -24...  
aus0001-0004  LINESTRING (153.2657 -24.70497, 153.26961 -24....  
...                                                         ...  
ber0002-0009  LINESTRING (-64.82204 32.25336, -64.82017 32.2...  
ber0002-0010  LINESTRING (-64.82143 32.25361, -64.82029 32.2...  
ber0002-0011  LINESTRING (-64.82118 32.25369, -64.82004 32.2...  
ber0002-0012  LINESTRING (-64.82049 32.25382, -64.82022 32.2...  
ber0002-0013  LINESTRING (-64.82022 32.25384, -64.81995 32.2...  

[147216 rows x 17 columns]

In [18]:
transects[transects.site_id.str.startswith("sar") & ~transects.trend.isna()]

site_id  orientation  along_dist  along_dist_norm  beach_slope  \
id                                                                             
sar0001-0000  sar0001          NaN         NaN              NaN          NaN   
sar0001-0001  sar0001          NaN         NaN              NaN          NaN   
sar0001-0002  sar0001          NaN         NaN              NaN          NaN   
sar0001-0003  sar0001          NaN         NaN              NaN          NaN   
sar0001-0004  sar0001          NaN         NaN              NaN          NaN   
...               ...          ...         ...              ...          ...   
sar2541-0000  sar2541          NaN         NaN              NaN          NaN   
sar2541-0001  sar2541          NaN         NaN              NaN          NaN   
sar2541-0002  sar2541          NaN         NaN              NaN          NaN   
sar2541-0003  sar2541          NaN         NaN              NaN          NaN   
sar2541-0004  sar2541          NaN         NaN              NaN          NaN   

              cil  ciu     trend  n_points  n_points_nonan  r2_score  \
id                                                                     
sar0001-0000  NaN  NaN -0.209107     671.0           667.0  0.003085   
sar0001-0001  NaN  NaN -0.168722     671.0           667.0  0.008700   
sar0001-0002  NaN  NaN -0.012318     671.0           669.0  0.000413   
sar0001-0003  NaN  NaN -0.043548     671.0           669.0  0.016188   
sar0001-0004  NaN  NaN  0.011825     671.0           669.0  0.001501   
...           ...  ...       ...       ...             ...       ...   
sar2541-0000  NaN  NaN -0.297976    1371.0          1371.0  0.203601   
sar2541-0001  NaN  NaN  0.011742    1371.0          1371.0  0.000671   
sar2541-0002  NaN  NaN  0.048766    1371.0          1371.0  0.008370   
sar2541-0003  NaN  NaN -0.212156    1371.0          1371.0  0.135324   
sar2541-0004  NaN  NaN -1.424914    1371.0          1371.0  0.750608   

                    mae          mse       rmse   intercept ERODIBILITY  \
id                                                                        
sar0001-0000  34.337637  2024.899412  44.998882  135.992639      Medium   
sar0001-0001  11.931958   460.730301  21.464629  197.877604      Medium   
sar0001-0002   5.801157    51.994338   7.210710  206.410952      Medium   
sar0001-0003   3.202754    16.331152   4.041182  239.500879      Medium   
sar0001-0004   2.692829    13.178444   3.630213  258.136097      Medium   
...                 ...          ...        ...         ...         ...   
sar2541-0000   5.662483    50.986351   7.140473  199.127124        High   
sar2541-0001   3.998349    30.150238   5.490923  165.933848        High   
sar2541-0002   4.829729    41.363000   6.431407  161.418975        High   
sar2541-0003   5.148677    42.220947   6.497765  182.397918        High   
sar2541-0004   7.903430    99.034737   9.951620  416.753361        High   

                                                     geometry  
id                                                             
sar0001-0000  LINESTRING (8.40852 38.86175, 8.40882 38.86535)  
sar0001-0001   LINESTRING (8.4084 38.86162, 8.41092 38.86464)  
sar0001-0002  LINESTRING (8.40893 38.86153, 8.41236 38.86393)  
sar0001-0003   LINESTRING (8.40904 38.86129, 8.41333 38.8626)  
sar0001-0004   LINESTRING (8.40904 38.86176, 8.4133 38.86037)  
...                                                       ...  
sar2541-0000  LINESTRING (8.85399 38.88006, 8.85388 38.87736)  
sar2541-0001  LINESTRING (8.85428 38.88005, 8.85417 38.87735)  
sar2541-0002  LINESTRING (8.85502 38.87993, 8.85398 38.87735)  
sar2541-0003   LINESTRING (8.85536 38.87984, 8.85418 38.8773)  
sar2541-0004   LINESTRING (8.85656 38.8791, 8.85349 38.87785)  

[19763 rows x 17 columns]

In [19]:
transects.drop_duplicates().to_file("transects_extended.geojson", driver="GeoJSON")